In [1]:
%matplotlib qt

import matplotlib.pyplot as plt
from poisson_fem import PoissonFEM
import ROM
import GenerativeSurrogate as gs
import Data as dta
import numpy as np
import scipy.sparse as sps
import scipy.sparse.linalg as lg
import time
import petsc4py
import sys
petsc4py.init(sys.argv)
from petsc4py import PETSc
import torch
from torch import optim

/home/constantin/anaconda3/envs/genDRROM/lib/python3.7/site-packages/tensorboard/compat/tensorflow_stub/dtypes.py:541: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/constantin/anaconda3/envs/genDRROM/lib/python3.7/site-packages/tensorboard/compat/tensorflow_stub/dtypes.py:542: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/constantin/anaconda3/envs/genDRROM/lib/python3.7/site-packages/tensorboard/compat/tensorflow_stub/dtypes.py:543: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/c

In [2]:
# Some fixed parameters
lin_dim_rom = 16                      # Linear number of rom elements
a = np.array([1, 2, 3])              # Boundary condition function coefficients
dim_z = 30                            # Latent space dimension
dtype = torch.float                  # Tensor data type

In [3]:
# Define mesh and boundary conditions
mesh = PoissonFEM.RectangularMesh(np.ones(lin_dim_rom)/lin_dim_rom)
# mesh.plot()

def origin(x):
    return np.abs(x[0]) < np.finfo(float).eps and np.abs(x[1]) < np.finfo(float).eps

def ess_boundary_fun(x):
    return 0.0
mesh.set_essential_boundary(origin, ess_boundary_fun)

def domain_boundary(x):
    # unit square
    return np.abs(x[0]) < np.finfo(float).eps or np.abs(x[1]) < np.finfo(float).eps or \
            np.abs(x[0]) > 1.0 - np.finfo(float).eps or np.abs(x[1]) > 1.0 - np.finfo(float).eps
mesh.set_natural_boundary(domain_boundary)

def flux(x):
    q = np.array([a[0] + a[2]*x[1], a[1] + a[2]*x[0]])
    return q

In [4]:
# Set up solver
ksp = PETSc.KSP().create()
ksp.setType('preonly')
precond = ksp.getPC()
precond.setType('cholesky')
ksp.setFromOptions() #???

In [5]:
#Spepify right hand side and stiffness matrix
#Define boundary flux field
rhs = PoissonFEM.RightHandSide(mesh)
rhs.set_natural_rhs(mesh, flux)
funSpace = PoissonFEM.FunctionSpace(mesh)
K = PoissonFEM.StiffnessMatrix(mesh, funSpace, ksp)
rhs.set_rhs_stencil(mesh, K)

In [6]:
# define rom
rom = ROM.ROM(mesh, K, rhs)

In [7]:
trainingData = dta.StokesData(range(256))
trainingData.read_data(['IMG'])
# trainingData.plotMicrostruct(1)
trainingData.reshape_microstructure_image()

In [8]:
model = gs.GenerativeSurrogate(rom, trainingData, dim_z)

In [9]:
steps = int(1)
for s in range(steps):
    print('step = ', s)
    batch_samples_pf = torch.multinomial(torch.ones(trainingData.n_samples_in), model.batch_size_N_pf)
#     batch_samples_pc = torch.multinomial(torch.ones(trainingData.n_samples_out), model.batch_size_N_pc)
    model.opt_latent_dist_step()
    model.pf_step(batch_samples_pf)
#     model.pc_step(batch_samples_pc)
    

step =  0
loss_pf =  tensor(11633023., grad_fn=<SubBackward0>)


In [10]:
# model.plot_input_reconstruction()
# f = plt.gcf()
# f.suptitle('Untrained, N = 1184', fontsize=32, y=.7)

In [11]:
xx, yy = np.meshgrid(np.linspace(0, 1, 129), np.linspace(0, 1, 129))

In [12]:
X = np.concatenate((np.expand_dims(xx.flatten(), axis=1), np.expand_dims(yy.flatten(), axis=1)), axis=1)

In [13]:
W = mesh.get_interpolation_matrix(X)

In [14]:
plt.contourf(W)

In [15]:
W.shape

(16641, 289)

RuntimeError: indices must be sparse_dim x nnz, but got: [3]

In [24]:
i = torch.LongTensor([[0, 1, 1], [2, 0, 2]])

In [25]:
v = torch.FloatTensor([3, 4, 5])

In [26]:
sp = torch.sparse.FloatTensor(i, v, torch.Size([2, 3]))

In [27]:
d = torch.sparse.FloatTensor(i, v, torch.Size([2, 3])).to_dense()

In [ ]:
%timeit torch.add(d, sp)

In [ ]:
%timeit d + d

In [ ]:
d

In [ ]:
not False

In [ ]:
vtx_list

In [17]:
mesh.scatter_matrix

tensor(indices=tensor([[  1,   2,   3,   4,   5,   6,   7,   8,   9,  10,  11,
                         12,  13,  14,  15,  16,  17,  18,  19,  20,  21,  22,
                         23,  24,  25,  26,  27,  28,  29,  30,  31,  32,  33,
                         34,  35,  36,  37,  38,  39,  40,  41,  42,  43,  44,
                         45,  46,  47,  48,  49,  50,  51,  52,  53,  54,  55,
                         56,  57,  58,  59,  60,  61,  62,  63,  64,  65,  66,
                         67,  68,  69,  70,  71,  72,  73,  74,  75,  76,  77,
                         78,  79,  80,  81,  82,  83,  84,  85,  86,  87,  88,
                         89,  90,  91,  92,  93,  94,  95,  96,  97,  98,  99,
                        100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110,
                        111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121,
                        122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132,
                        133, 134, 135, 136, 137, 138

In [38]:
u = torch.ones((288, 1))
%timeit torch.sparse.mm(mesh.scatter_matrix, u)

14.3 µs ± 42.6 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


In [39]:
sc = mesh.scatter_matrix.to_dense()

In [40]:
%timeit sc @ u

10.7 µs ± 51.8 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


In [41]:
sc

tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [1., 0., 0.,  ..., 0., 0., 0.],
        [0., 1., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 1., 0., 0.],
        [0., 0., 0.,  ..., 0., 1., 0.],
        [0., 0., 0.,  ..., 0., 0., 1.]])

In [42]:
import scipy.sparse as sps
sc = sps.csr_matrix(sc)

In [43]:
sc

<289x288 sparse matrix of type '<class 'numpy.float32'>'
	with 288 stored elements in Compressed Sparse Row format>

In [44]:
scp = PETSc.Mat().createAIJ(size=sc.shape, csr=(sc.indptr, sc.indices, sc.data))

In [45]:
up = PETSc.Vec().createWithArray(u.detach().numpy())

In [47]:
y = PETSc.Vec().createSeq(289)

In [48]:
%timeit scp.mult(up, y)

570 ns ± 9.49 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)


In [50]:
y[5] = 3

In [51]:
y[5]

array(3.)

In [52]:
y